
<h1 style="text-align: center;"> Computer Vision Laboratory Assignment 8 </h1>
<h2 style="text-align: center;"> 122CS0067 Amiya Chowdhury 20/03/2025 </h2>

<p  center>1. Implement convolutional neural network (CNN) models with following specifications
using TensorFlow for classifying the MNIST dataset. Train the model on the MNIST
training set and evaluate its performance on the test set. Write modularized code and
call it 5 times and compute the mean of test accuracy for each of the following models
[Model-1, Model-2, Model-3, Model-4].</p>

In [1]:
#importing the libraries
import tensorflow as tf
import numpy as np
from tensorflow.keras import utils,models,layers
import seaborn as sns
from tensorflow.keras.datasets import mnist
from tensorflow.keras.initializers import RandomNormal

import keras
from keras import optimizers
from keras.models import Model
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, AveragePooling2D
from sklearn.ensemble import RandomForestClassifier
import os
import glob as gb
import cv2

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# the data, shuffled and split between train and test sets and normalized
def load_data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train/255
    X_test = X_test/255
    return (X_train, y_train), (X_test, y_test)

<p center>a. Model-1: Add a convolution layer with 32 3 × 3 filters with stride 2 and relu
activation. Add a maxpooling layer with kernel size 2 × 2 with stride 1. Add a
convolution layer with 16 4 × 4 filters with stride 2 and relu activation. Add a
maxpooling layer with kernel size 4 × 4 with stride 2. Flatten the output and
add a fully connected layer with 8 neurons with relu activation. Add a fully
connected layer with 10 neurons and softmax activation.</p>
<p>Extract the features from second last fully connected layer (having 8 neurons)
and model it using a Random Forest classifier. Use Adam optimizer with
batch size 128, learning rate 0.01 and epochs set to 5.</p>

In [3]:
# Define Model-1
def model1():
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2), strides=1))
    model.add(layers.Conv2D(16, (4, 4), strides=2, activation='relu'))
    model.add(layers.MaxPooling2D((4, 4), strides=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(8, activation='relu', name='feature_layer_1'))  # Feature extraction layer
    model.add(layers.Dense(10, activation='softmax'))
    model.summary()
    return model

In [4]:
# Train and extract features
def train_evaluate_model1():
    (X_train, y_train), (X_test, y_test) = load_data()
    model = model1()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1,validation_data=(X_test,y_test))
    #extracting deep features
    feature_extractor = Model(inputs=model.input, outputs=model.get_layer('feature_layer_1').output)
    features_train = feature_extractor.predict(X_train)
    features_test = feature_extractor.predict(X_test)
    
    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(features_train, y_train)
    y_pred = rf_classifier.predict(features_test)
    rf_accuracy=accuracy_score(y_test, y_pred)
    return rf_accuracy


In [39]:
mean = 0
loop_count = 5
for _ in range(loop_count):
    mean += train_evaluate_model1()
mean /= loop_count
print(f"Mean for 1.a is {mean}")

Model: "sequential_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_98 (Conv2D)          (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_71 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_99 (Conv2D)          (None, 5, 5, 16)          8208      
                                                                 
 max_pooling2d_72 (MaxPooli  (None, 1, 1, 16)          0         
 ng2D)                                                           
                                                                 
 flatten_49 (Flatten)        (None, 16)                0         
                                                                 
 feature_layer_1 (Dense)     (None, 8)               

                                                                 
Total params: 8754 (34.20 KB)
Trainable params: 8754 (34.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
469/469 [==============================] - 9s 18ms/step - loss: 1.0725 - accuracy: 0.6289 - val_loss: 0.6718 - val_accuracy: 0.7808
Epoch 2/5
469/469 [==============================] - 8s 17ms/step - loss: 0.5646 - accuracy: 0.8326 - val_loss: 0.4838 - val_accuracy: 0.8636
Epoch 3/5
469/469 [==============================] - 8s 17ms/step - loss: 0.4726 - accuracy: 0.8623 - val_loss: 0.4410 - val_accuracy: 0.8670
Epoch 4/5
469/469 [==============================] - 8s 17ms/step - loss: 0.4357 - accuracy: 0.8727 - val_loss: 0.3833 - val_accuracy: 0.8925
Epoch 5/5
313/313 [==============================] - 1s 2ms/step
Model: "sequential_53"
_________________________________________________________________
 Layer (type)                Output Shape     

<p center>b. Model-2: Add a convolution layer with 32 3 × 3 filters with stride 2 and relu
activation. Add an average pooling layer with kernel size 2 × 2 with stride 1.
Add a convolution layer with 16 4 × 4 filters with stride 2 and relu activation.
Add an average pooling layer with kernel size 4 × 4 with stride 2. Flatten the
output and add a fully connected layer with 8 neurons with relu activation.
Add a fully connected layer with 10 neurons and softmax activation.>Add a fully connected layer with 10 neurons and softmax activation.</p>
<p center>
Extract the features from second last fully connected layer (having 8 neurons)
and model it using a Random Forest classifier. Use Adam optimizer with
batch size 128, learning rate 0.01 and epochs set to 5.</p>

In [6]:
# Define Model-2
def model2():
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2), strides=1))
    model.add(layers.Conv2D(16, (4, 4), strides=2, activation='relu'))
    model.add(layers.AveragePooling2D((4, 4), strides=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(8, activation='relu', name='feature_layer_1'))  # Feature extraction layer
    model.add(layers.Dense(10, activation='softmax'))
    model.summary()
    return model

In [7]:
def train_evaluate_model2():
    (X_train, y_train), (X_test, y_test) = load_data()
    model = model2()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1,validation_data=(X_test,y_test))
    #extracting deep features
    feature_extractor = Model(inputs=model.input, outputs=model.get_layer('feature_layer_1').output)
    features_train = feature_extractor.predict(X_train)
    features_test = feature_extractor.predict(X_test)
    
    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(features_train, y_train)
    y_pred = rf_classifier.predict(features_test)
    rf_accuracy=accuracy_score(y_test, y_pred)
    return rf_accuracy

In [9]:
mean2 = 0
loop_count = 5
for _ in range(loop_count):
    mean2 += train_evaluate_model2()
mean2 /= loop_count
print(f"Mean for 1.b is {mean2}")

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 5, 5, 16)          8208      
                                                                 
 average_pooling2d_5 (Avera  (None, 1, 1, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten_10 (Flatten)        (None, 16)                0         
                                                                 
 feature_layer_1 (Dense)     (None, 8)               

                                                                 
Total params: 8754 (34.20 KB)
Trainable params: 8754 (34.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
469/469 [==============================] - 9s 17ms/step - loss: 1.9245 - accuracy: 0.2495 - val_loss: 1.7063 - val_accuracy: 0.2889
Epoch 2/5
469/469 [==============================] - 8s 16ms/step - loss: 1.6585 - accuracy: 0.3236 - val_loss: 1.5967 - val_accuracy: 0.3399
Epoch 3/5
469/469 [==============================] - 8s 16ms/step - loss: 1.5826 - accuracy: 0.3591 - val_loss: 1.5396 - val_accuracy: 0.3786
Epoch 4/5
469/469 [==============================] - 8s 17ms/step - loss: 1.5251 - accuracy: 0.3718 - val_loss: 1.5449 - val_accuracy: 0.3907
Epoch 5/5
313/313 [==============================] - 1s 2ms/step
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape     

<p center>c. Model-3: Extract the deep features from Model-1 and Model-2 stack the
features horizontally and model it using a Random Forest classifier. </p>

In [11]:
def train_evaluate_model3():
    (X_train, y_train), (X_test, y_test) = load_data()
    model = model2()
    model_= model1()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model_.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1,validation_data=(X_test,y_test))
    model_.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1,validation_data=(X_test,y_test))
    #extracting deep features
    feature_extractor = Model(inputs=model.input, outputs=model.get_layer('feature_layer_1').output)
    feature_extractor_= Model(inputs=model_.input, outputs=model_.get_layer('feature_layer_1').output)
    
    #Train features Stack
    features1 = feature_extractor.predict(X_train)
    features2 = feature_extractor_.predict(X_train)
    features_train = np.hstack((features1,features2))
    #Test features Stack
    features1_ = feature_extractor.predict(X_test)
    features2_ = feature_extractor_.predict(X_test)
    features_test = np.hstack((features1_,features2_))
    
    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(features_train, y_train)
    y_pred = rf_classifier.predict(features_test)
    rf_accuracy=accuracy_score(y_test, y_pred)
    return rf_accuracy

In [12]:
mean3 = 0
loop_count = 5
for _ in range(loop_count):
    mean3 += train_evaluate_model3()
mean3 /= loop_count
print(f"Mean for 1.c is {mean3}")

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_31 (Conv2D)          (None, 5, 5, 16)          8208      
                                                                 
 average_pooling2d_10 (Aver  (None, 1, 1, 16)          0         
 agePooling2D)                                                   
                                                                 
 flatten_15 (Flatten)        (None, 16)                0         
                                                                 
 feature_layer_1 (Dense)     (None, 8)               

469/469 [==============================] - 8s 16ms/step - loss: 0.3719 - accuracy: 0.8823 - val_loss: 0.3234 - val_accuracy: 0.9005
Epoch 5/5
469/469 [==============================] - 8s 16ms/step - loss: 0.3408 - accuracy: 0.8934 - val_loss: 0.3228 - val_accuracy: 0.8990
Epoch 1/5
469/469 [==============================] - 9s 17ms/step - loss: 0.8050 - accuracy: 0.7207 - val_loss: 0.4697 - val_accuracy: 0.8530
Epoch 2/5
469/469 [==============================] - 8s 17ms/step - loss: 0.3998 - accuracy: 0.8779 - val_loss: 0.3323 - val_accuracy: 0.8947
Epoch 3/5
469/469 [==============================] - 8s 17ms/step - loss: 0.3194 - accuracy: 0.9047 - val_loss: 0.3112 - val_accuracy: 0.9030
Epoch 4/5
469/469 [==============================] - 8s 17ms/step - loss: 0.2822 - accuracy: 0.9151 - val_loss: 0.2431 - val_accuracy: 0.9255
Epoch 5/5
313/313 [==============================] - 1s 2ms/step
Model: "sequential_19"
_________________________________________________________________
 Lay

                                                                 
 flatten_22 (Flatten)        (None, 16)                0         
                                                                 
 feature_layer_1 (Dense)     (None, 8)                 136       
                                                                 
 dense_22 (Dense)            (None, 10)                90        
                                                                 
Total params: 8754 (34.20 KB)
Trainable params: 8754 (34.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
469/469 [==============================] - 9s 17ms/step - loss: 1.2324 - accuracy: 0.5454 - val_loss: 0.7568 - val_accuracy: 0.7355
Epoch 2/5
469/469 [==============================] - 8s 17ms/step - loss: 0.7107 - accuracy: 0.7545 - val_loss: 0.5725 - val_accuracy: 0.8177
Epoch 3/5
469/469 [==============================] - 8s 17ms/step - loss: 0.5936 - accur

<p center>d.  Model-4: Extract the deep features from Model-1 and Model-2 stack the
features horizontally, reduce the dimension to either 8, 10 or 12 using
principal component analysis (PCA) and model the reduced features using a
Random Forest classifier. Identify the best number of reduced components of
PCA.</p>

In [29]:
from sklearn.decomposition import PCA

def train_evaluate_model4(comp):
    (X_train, y_train), (X_test, y_test) = load_data()
    model = model2()
    model_= model1()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model_.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1,validation_data=(X_test,y_test))
    model_.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1,validation_data=(X_test,y_test))
    #extracting deep features
    feature_extractor = Model(inputs=model.input, outputs=model.get_layer('feature_layer_1').output)
    feature_extractor_= Model(inputs=model_.input, outputs=model_.get_layer('feature_layer_1').output)
    
    #Train features Stack
    features1 = feature_extractor.predict(X_train)
    features2 = feature_extractor_.predict(X_train)
    features_train = np.hstack((features1,features2))
    #Test features Stack
    features1_ = feature_extractor.predict(X_test)
    features2_ = feature_extractor_.predict(X_test)
    features_test = np.hstack((features1_,features2_))
    
    #PCA Decomposition
    rf_accuracy=np.array([])
    for i in comp:
        final_train = PCA(n_components=i).fit_transform(features_train)
        final_test = PCA(n_components=i).fit_transform(features_test)

        rf_classifier = RandomForestClassifier()
        rf_classifier.fit(final_train, y_train)
        y_pred = rf_classifier.predict(final_test)
        rf_accuracy=np.append(rf_accuracy,accuracy_score(y_test, y_pred))
    return rf_accuracy

In [48]:
(X_train, y_train), (X_test, y_test) = load_data()
model = model2()
model_= model1()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
model_.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1,validation_data=(X_test,y_test))
model_.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1,validation_data=(X_test,y_test))
    #extracting deep features
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('feature_layer_1').output)
feature_extractor_= Model(inputs=model_.input, outputs=model_.get_layer('feature_layer_1').output)
    
    #Train features Stack
features1 = feature_extractor.predict(X_train)
features2 = feature_extractor_.predict(X_train)
features_train = np.hstack((features1,features2))
    #Test features Stack
features1_ = feature_extractor.predict(X_test)
features2_ = feature_extractor_.predict(X_test)
features_test = np.hstack((features1_,features2_))

Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_108 (Conv2D)         (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_81 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_109 (Conv2D)         (None, 5, 5, 16)          8208      
                                                                 
 average_pooling2d_27 (Aver  (None, 1, 1, 16)          0         
 agePooling2D)                                                   
                                                                 
 flatten_54 (Flatten)        (None, 16)                0         
                                                                 
 feature_layer_1 (Dense)     (None, 8)               

In [36]:
def model4_eval(features_train,features_test,comp):
    rf_accuracy=np.array([])
    for i in comp:
        final_train = PCA(n_components=i).fit_transform(features_train)
        final_test = PCA(n_components=i).fit_transform(features_test)

        rf_classifier = RandomForestClassifier()
        rf_classifier.fit(final_train, y_train)
        y_pred = rf_classifier.predict(final_test)
        rf_accuracy=np.append(rf_accuracy,accuracy_score(y_test, y_pred))
    return rf_accuracy

In [38]:
#extracts features once, runs the stochastic models multiple times
mean4 = np.array([0,0,0],dtype='float64')
loop_count = 5
comp=[8,10,12]
for _ in range(loop_count):
    mean4 += model4_eval(features_train,features_test,comp)
mean4 /= loop_count
print(f"Mean for 1.d is :")
for i in range(len(mean4)):
    print(f"\t PCA {comp[i]}={mean4[i]}")

Mean for 1.d is :
	 PCA 8=0.9516600000000001
	 PCA 10=0.9271799999999999
	 PCA 12=0.9265599999999999


In [30]:
#trains the underlying model as well (conv model)
mean4 = np.array([0,0,0],dtype='float64')
loop_count = 5
comp=[8,10,12]
for _ in range(loop_count):
    mean4 += train_evaluate_model4(comp)
mean4 /= loop_count
print(f"Mean for 1.d is :")
for i in range(len(mean4)):
    print(f"\t PCA {comp[i]}={mean4[i]}")

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_74 (Conv2D)          (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_53 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_75 (Conv2D)          (None, 5, 5, 16)          8208      
                                                                 
 average_pooling2d_21 (Aver  (None, 1, 1, 16)          0         
 agePooling2D)                                                   
                                                                 
 flatten_37 (Flatten)        (None, 16)                0         
                                                                 
 feature_layer_1 (Dense)     (None, 8)               

469/469 [==============================] - 8s 17ms/step - loss: 0.2213 - accuracy: 0.9323 - val_loss: 0.1837 - val_accuracy: 0.9434
Epoch 5/5
469/469 [==============================] - 8s 17ms/step - loss: 0.1993 - accuracy: 0.9399 - val_loss: 0.1697 - val_accuracy: 0.9474
Epoch 1/5
469/469 [==============================] - 9s 17ms/step - loss: 0.9086 - accuracy: 0.6755 - val_loss: 0.4997 - val_accuracy: 0.8426
Epoch 2/5
469/469 [==============================] - 8s 17ms/step - loss: 0.4639 - accuracy: 0.8549 - val_loss: 0.3642 - val_accuracy: 0.8937
Epoch 3/5
469/469 [==============================] - 8s 17ms/step - loss: 0.3553 - accuracy: 0.8908 - val_loss: 0.3533 - val_accuracy: 0.8871
Epoch 4/5
469/469 [==============================] - 8s 17ms/step - loss: 0.3038 - accuracy: 0.9075 - val_loss: 0.2891 - val_accuracy: 0.9105
Epoch 5/5
313/313 [==============================] - 1s 2ms/step
Model: "sequential_41"
_________________________________________________________________
 Lay

                                                                 
 flatten_44 (Flatten)        (None, 16)                0         
                                                                 
 feature_layer_1 (Dense)     (None, 8)                 136       
                                                                 
 dense_44 (Dense)            (None, 10)                90        
                                                                 
Total params: 8754 (34.20 KB)
Trainable params: 8754 (34.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
469/469 [==============================] - 9s 18ms/step - loss: 1.2080 - accuracy: 0.5821 - val_loss: 0.7280 - val_accuracy: 0.7663
Epoch 2/5
469/469 [==============================] - 8s 17ms/step - loss: 0.6384 - accuracy: 0.7929 - val_loss: 0.5380 - val_accuracy: 0.8272
Epoch 3/5
469/469 [==============================] - 8s 17ms/step - loss: 0.5420 - accur

In [31]:
print('Training takes a long time :(')

Training takes a long time :(


<p center>e. Draw conclusions on the best model among the above four models for
classifying MNIST dataset. </p>

In [41]:
print("The mean accuracies of the models are as follows:")
print(f"Mean for 1.a is {mean}")
print(f"Mean for 1.b is {mean2}")
print(f"Mean for 1.c is {mean3}")
print(f"Mean for 1.d is :")
for i in range(len(mean4)):
    print(f"\t PCA {comp[i]}={mean4[i]}")

The mean accuracies of the models are as follows:
Mean for 1.a is 0.90442
Mean for 1.b is 0.8041
Mean for 1.c is 0.95536
Mean for 1.d is :
	 PCA 8=0.9516600000000001
	 PCA 10=0.9271799999999999
	 PCA 12=0.9265599999999999


In [33]:
print("The best model appears to be Model3, in terms of the accuracy metric..")

The best model appears to be Model3, in terms of the accuracy metric..


f. Replace PCA with ICA in Model-4 and compare performance 

In [44]:
from sklearn.decomposition import FastICA
def model4_evalICA(features_train,features_test,comp):
    rf_accuracy=np.array([])
    for i in comp:
        final_train = FastICA(n_components=i).fit_transform(features_train)
        final_test = FastICA(n_components=i).fit_transform(features_test)

        rf_classifier = RandomForestClassifier()
        rf_classifier.fit(final_train, y_train)
        y_pred = rf_classifier.predict(final_test)
        rf_accuracy=np.append(rf_accuracy,accuracy_score(y_test, y_pred))
    return rf_accuracy

In [51]:
#extracts features once, runs the stochastic models multiple times
mean5 = np.array([0,0,0],dtype='float64')
loop_count = 5
comp=[8,10,12]
for _ in range(loop_count):
    mean5 += model4_evalICA(features_train,features_test,comp)
mean5 /= loop_count
print(f"Mean for 1.f is :")
for i in range(len(mean4)):
    print(f"\t ICA {comp[i]}={mean4[i]}")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: divide by zero encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: invalid value encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: divide by zero encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: invalid value encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: divide by zero encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastic

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: divide by zero encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: invalid value encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: divide by zero encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: invalid value encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:532: RuntimeWarning: divide by zero encountered in divide
  K = (u / d).T[:n_components]  # see (6.33) p.140
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastic

Mean for 1.f is :
	 ICA 8=0.0318
	 ICA 10=0.1682
	 ICA 12=0.0995


In [55]:
def model1_dr():
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=2, activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2), strides=1))
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(layers.Conv2D(16, (4, 4), strides=2, activation='relu'))
    model.add(layers.MaxPooling2D((4, 4), strides=2))
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(8, activation='relu', name='feature_layer_1'))  # Feature extraction layer
    model.add(layers.Dense(10, activation='softmax'))
    model.summary()
    return model
              # Train and extract features
def train_evaluate_model1_dr():
    (X_train, y_train), (X_test, y_test) = load_data()
    model = model1_dr()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1,validation_data=(X_test,y_test))
    #extracting deep features
    feature_extractor = Model(inputs=model.input, outputs=model.get_layer('feature_layer_1').output)
    features_train = feature_extractor.predict(X_train)
    features_test = feature_extractor.predict(X_test)
    
    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(features_train, y_train)
    y_pred = rf_classifier.predict(features_test)
    rf_accuracy=accuracy_score(y_test, y_pred)
    return rf_accuracy
mean = 0
loop_count = 5
for _ in range(loop_count):
    mean += train_evaluate_model1_dr()
mean /= loop_count
print(f"Mean for 1.a with 0.2 Dropout is {mean}")

Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_112 (Conv2D)         (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_84 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 conv2d_113 (Conv2D)         (None, 5, 5, 16)          8208      
                                                                 
 max_pooling2d_85 (MaxPooli  (None, 1, 1, 16)          0         
 ng2D)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 1, 1, 16)        

 ng2D)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 12, 12, 32)        0         
                                                                 
 conv2d_119 (Conv2D)         (None, 5, 5, 16)          8208      
                                                                 
 max_pooling2d_91 (MaxPooli  (None, 1, 1, 16)          0         
 ng2D)                                                           
                                                                 
 dropout_7 (Dropout)         (None, 1, 1, 16)          0         
                                                                 
 flatten_59 (Flatten)        (None, 16)                0         
                                                                 
 feature_layer_1 (Dense)     (None, 8)                 136       
                                                                 
 dense_59 